In [9]:
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd

from main import main

from query import DataLoader

load_dotenv()

groq_key = os.environ['groqkey3']

data_path = Path(os.environ['data_path'])


In [10]:
#already run 

prev_df = pd.read_csv(data_path / 'total_results.csv')

In [11]:
dl = DataLoader(data_path)

labeled_df = dl.get_label_df()
#remove already labeled
labeled_df = labeled_df[~labeled_df.key.isin(prev_df.key)]
#labeled_df = labeled_df.sample(10)
#labeled_df = labeled_df[labeled_df.key == 'SyntheticPt1']

In [12]:
notes = pd.read_csv(dl.data_path / "labled_notes_w_summary.csv")

In [13]:
for index, row in labeled_df.iterrows():
    encounter_key = row["key"]
    print("label_index:",index," key:",encounter_key)
    ret_recommendation, ret_reasoning, token_usage, search_history_so_far, token_count_history = main(
        groq_key=groq_key, data_path=data_path, encounter_key=encounter_key
    )
    print(ret_recommendation)
    labeled_df.at[index, 'ret_recommendation'] = ret_recommendation
    labeled_df.at[index, 'ret_reasoning'] = ret_reasoning
    labeled_df.at[index, 'token_usage'] = token_usage
    labeled_df.at[index, 'search_history_so_far'] = str(search_history_so_far)
    labeled_df.at[index, 'token_count_history'] = str(token_count_history)



label_index: 36  key: D5A533360225FF
deprescribe
label_index: 38  key: SyntheticPt2
continue
label_index: 39  key: SyntheticPt3
stop
label_index: 40  key: SyntheticPt4
deprescribe
label_index: 41  key: SyntheticPt5
deprescribe
label_index: 42  key: SyntheticPt6
no notes found
deprescribe


In [17]:
labeled_df = labeled_df[labeled_df.token_usage > 0]

In [18]:
labeled_df.to_csv(dl.data_path / 'sample_results.csv', index=False)

In [19]:
#append the new sample_results.csv to the total_results.csv
labeled_df = pd.concat([prev_df, labeled_df])

labeled_df.to_csv(dl.data_path / 'total_results.csv', index=False)

In [6]:
from eval import evaluate_multiclass_classification, get_best_diagnosis_match

y_true = labeled_df['recommendation']
y_pred = labeled_df['ret_recommendation']
class_labels = ['continue', 'deprescribe', 'stop']

# Evaluate the model
metrics = evaluate_multiclass_classification(y_true, y_pred, class_labels)

ValueError: Number of classes, 1, does not match size of target_names, 3. Try specifying the labels parameter

In [ ]:
labeled_df['diagnosis_key'] = labeled_df['diagnosis'].str.split('-').str[0]

/var/folders/8w/6bqg5xh14393yv1z0lhtm2z40000gn/T/ipykernel_57421/3775046982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_df['diagnosis_key'] = labeled_df['diagnosis'].str.split('-').str[0]


In [12]:
for index, row in labeled_df.iterrows():
    matched_diagnosis_key, matched_diagnosis_label = get_best_diagnosis_match(row.ret_reasoning)
    labeled_df.at[index, 'matched_diagnosis_key'] = matched_diagnosis_key
    labeled_df.at[index, 'matched_diagnosis_label'] = matched_diagnosis_label
    